In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sklearn.linear_model

Step 1:Grab the dataset



> Search for missing values



In [ ]:
data=pd.read_csv('/content/modified_data.csv')

In [ ]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


Step Two:Prepare Dataset

In [ ]:
data.drop('Id',axis=1,inplace=True)

In [ ]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
data.shape

(1460, 76)

In [ ]:
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage       True
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool

In [ ]:
#Get all col
data.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'WoodDeckSF'

In [ ]:
original_features=list(data.columns)
original_features

['MSSubClass',
 'MSZoning',
 'LotFrontage',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch

In [ ]:
features_with_missing=list(data.columns[data.isna().any()])
features_with_missing
len(features_with_missing)

15

In [ ]:
len(data)

1460

In [ ]:
for feature in features_with_missing:
  print(feature,'',(data[feature].isna().sum())/len(data)*100)

LotFrontage  17.73972602739726
MasVnrType  0.547945205479452
MasVnrArea  0.547945205479452
BsmtQual  2.5342465753424657
BsmtCond  2.5342465753424657
BsmtExposure  2.6027397260273974
BsmtFinType1  2.5342465753424657
BsmtFinType2  2.6027397260273974
Electrical  0.0684931506849315
FireplaceQu  47.26027397260274
GarageType  5.5479452054794525
GarageYrBlt  5.5479452054794525
GarageFinish  5.5479452054794525
GarageQual  5.5479452054794525
GarageCond  5.5479452054794525


In [ ]:
#Axis 0=row,axis 1=column (0,4 threshhold-drop values with 40% and above missing will be dropped)
data.dropna(axis=1,thresh=0.5*(len(data)),inplace=True)

In [ ]:
mean_fill=['LotFrontage','MasVnrArea']
bfill=['FireplaceQu']
ffill=list(set(features_with_missing)-set(mean_fill)-set(bfill))

In [ ]:
ffill

['BsmtFinType1',
 'BsmtExposure',
 'MasVnrType',
 'Electrical',
 'GarageYrBlt',
 'GarageFinish',
 'GarageType',
 'GarageQual',
 'BsmtCond',
 'BsmtFinType2',
 'GarageCond',
 'BsmtQual']

In [ ]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [ ]:
for feature in ffill:
  data[feature].fillna(method='ffill',inplace=True)

In [ ]:
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [ ]:
data.isna().any()

MSSubClass       False
MSZoning         False
LotFrontage      False
LotArea          False
Street           False
                 ...  
MoSold           False
YrSold           False
SaleType         False
SaleCondition    False
SalePrice        False
Length: 76, dtype: bool



> Encode the dataset



In [ ]:
cat_data=data.select_dtypes(include=['object']).copy()
cat_data.head(2)

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal


In [ ]:
cat_features=list(cat_data.columns)
cat_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
nominal=['MSZoning','LandContour','LotConfig','Neighborhood']
ordinal=list(set(cat_features)-set(nominal))
numerical=list(set(original_features)-set(cat_features))
target=['SalePrice']

In [ ]:
data[target]

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [ ]:
for feature in ordinal:
  data[feature]=(data[feature].astype('category')).cat.codes

In [ ]:
data[ordinal]

,SaleCondition,Heating,Street,Exterior1st,GarageType,BsmtQual,Functional,KitchenQual,ExterCond,BsmtFinType1,BsmtExposure,BldgType,Exterior2nd,HeatingQC,Electrical,ExterQual,BsmtFinType2,GarageCond,FireplaceQu,RoofMatl,Condition2,Foundation,MasVnrType,Utilities,BsmtCond,CentralAir,LotShape,Condition1,RoofStyle,GarageFinish,GarageQual,HouseStyle,SaleType,LandSlope,PavedDrive
0,4,1,1,12,1,2,6,2,4,2,3,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,3,2,1,1,4,5,8,0,2
1,4,1,1,8,1,2,6,3,4,0,1,0,8,0,4,3,5,4,4,1,2,1,2,0,3,1,3,1,1,1,4,2,8,0,2
2,4,1,1,12,1,2,6,2,4,2,2,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,0,2,1,1,4,5,8,0,2
3,0,1,1,13,5,3,6,2,4,0,3,0,15,2,4,3,5,4,2,1,2,0,2,0,1,1,0,2,1,2,4,5,8,0,2
4,4,1,1,12,1,2,6,2,4,2,0,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,0,2,1,1,4,5,8,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,4,1,1,12,1,2,6,3,4,5,3,0,13,0,4,3,5,4,4,1,2,2,2,0,3,1,3,2,1,1,4,5,8,0,2
1456,4,1,1,9,1,2,2,3,4,0,3,0,10,4,4,3,4,4,4,1,2,1,3,0,3,1,3,2,1,2,4,2,8,0,2
1457,4,1,1,5,1,3,6,2,2,2,3,0,5,0,4,0,5,4,2,1,2,4,2,0,1,1,3,2,1,1,4,5,8,0,2
1458,4,1,1,8,1,3,6,2,4,2,2,0,8,2,0,3,4,4,-1,1,2,1,2,0,3,1,3,2,3,2,4,2,8,0,2


In [ ]:
#One hot encoding
df_nominal=pd.get_dummies(data[nominal])

In [ ]:
df_ordinal=data[ordinal]

In [ ]:
df_numerical=data[numerical]


> Create a new dataset



In [ ]:
encoded_data=pd.concat([df_numerical,df_nominal,df_ordinal],axis=1)

In [ ]:
encoded_data.head(3)

,WoodDeckSF,MoSold,BedroomAbvGr,SalePrice,OverallQual,BsmtFinSF2,YrSold,EnclosedPorch,MasVnrArea,GarageArea,LotFrontage,OpenPorchSF,1stFlrSF,Fireplaces,FullBath,BsmtHalfBath,BsmtFinSF1,ScreenPorch,BsmtUnfSF,PoolArea,MSSubClass,TotalBsmtSF,2ndFlrSF,HalfBath,OverallCond,GrLivArea,YearBuilt,LowQualFinSF,YearRemodAdd,BsmtFullBath,GarageCars,3SsnPorch,TotRmsAbvGrd,LotArea,GarageYrBlt,KitchenAbvGr,MiscVal,MSZoning_C (all),MSZoning_FV,MSZoning_RH,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SaleCondition,Heating,Street,Exterior1st,GarageType,BsmtQual,Functional,KitchenQual,ExterCond,BsmtFinType1,BsmtExposure,BldgType,Exterior2nd,HeatingQC,Electrical,ExterQual,BsmtFinType2,GarageCond,FireplaceQu,RoofMatl,Condition2,Foundation,MasVnrType,Utilities,BsmtCond,CentralAir,LotShape,Condition1,RoofStyle,GarageFinish,GarageQual,HouseStyle,SaleType,LandSlope,PavedDrive
0,0,2,3,208500,7,0,2008,0,196.0,548,65.0,61,856,0,2,0,706,0,150,0,60,856,854,1,5,1710,2003,0,2003,1,2,0,8,8450,2003.0,1,0,0,0,0,...,0,0,0,0,0,4,1,1,12,1,2,6,2,4,2,3,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,3,2,1,1,4,5,8,0,2
1,298,5,3,181500,6,0,2007,0,0.0,460,80.0,0,1262,1,2,1,978,0,284,0,20,1262,0,0,8,1262,1976,0,1976,0,2,0,6,9600,1976.0,1,0,0,0,0,...,0,0,0,0,1,4,1,1,8,1,2,6,3,4,0,1,0,8,0,4,3,5,4,4,1,2,1,2,0,3,1,3,1,1,1,4,2,8,0,2
2,0,9,3,223500,7,0,2008,0,162.0,608,68.0,42,920,1,2,0,486,0,434,0,60,920,866,1,5,1786,2001,0,2002,1,2,0,6,11250,2001.0,1,0,0,0,0,...,0,0,0,0,0,4,1,1,12,1,2,6,2,4,2,2,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,0,2,1,1,4,5,8,0,2


In [ ]:
encoded_data.shape

(1460, 111)

Data Normalization

In [ ]:
encoded_data[target]

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [ ]:
X=encoded_data.drop('SalePrice',axis=1)

In [ ]:
X

,WoodDeckSF,MoSold,BedroomAbvGr,OverallQual,BsmtFinSF2,YrSold,EnclosedPorch,MasVnrArea,GarageArea,LotFrontage,OpenPorchSF,1stFlrSF,Fireplaces,FullBath,BsmtHalfBath,BsmtFinSF1,ScreenPorch,BsmtUnfSF,PoolArea,MSSubClass,TotalBsmtSF,2ndFlrSF,HalfBath,OverallCond,GrLivArea,YearBuilt,LowQualFinSF,YearRemodAdd,BsmtFullBath,GarageCars,3SsnPorch,TotRmsAbvGrd,LotArea,GarageYrBlt,KitchenAbvGr,MiscVal,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,...,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SaleCondition,Heating,Street,Exterior1st,GarageType,BsmtQual,Functional,KitchenQual,ExterCond,BsmtFinType1,BsmtExposure,BldgType,Exterior2nd,HeatingQC,Electrical,ExterQual,BsmtFinType2,GarageCond,FireplaceQu,RoofMatl,Condition2,Foundation,MasVnrType,Utilities,BsmtCond,CentralAir,LotShape,Condition1,RoofStyle,GarageFinish,GarageQual,HouseStyle,SaleType,LandSlope,PavedDrive
0,0,2,3,7,0,2008,0,196.0,548,65.0,61,856,0,2,0,706,0,150,0,60,856,854,1,5,1710,2003,0,2003,1,2,0,8,8450,2003.0,1,0,0,0,0,1,...,0,0,0,0,0,4,1,1,12,1,2,6,2,4,2,3,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,3,2,1,1,4,5,8,0,2
1,298,5,3,6,0,2007,0,0.0,460,80.0,0,1262,1,2,1,978,0,284,0,20,1262,0,0,8,1262,1976,0,1976,0,2,0,6,9600,1976.0,1,0,0,0,0,1,...,0,0,0,0,1,4,1,1,8,1,2,6,3,4,0,1,0,8,0,4,3,5,4,4,1,2,1,2,0,3,1,3,1,1,1,4,2,8,0,2
2,0,9,3,7,0,2008,0,162.0,608,68.0,42,920,1,2,0,486,0,434,0,60,920,866,1,5,1786,2001,0,2002,1,2,0,6,11250,2001.0,1,0,0,0,0,1,...,0,0,0,0,0,4,1,1,12,1,2,6,2,4,2,2,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,0,2,1,1,4,5,8,0,2
3,0,2,3,7,0,2006,272,0.0,642,60.0,35,961,1,1,0,216,0,540,0,70,756,756,0,5,1717,1915,0,1970,1,3,0,7,9550,1998.0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,1,13,5,3,6,2,4,0,3,0,15,2,4,3,5,4,2,1,2,0,2,0,1,1,0,2,1,2,4,5,8,0,2
4,192,12,4,8,0,2008,0,350.0,836,84.0,84,1145,1,2,0,655,0,490,0,60,1145,1053,1,5,2198,2000,0,2000,1,3,0,9,14260,2000.0,1,0,0,0,0,1,...,0,0,0,0,0,4,1,1,12,1,2,6,2,4,2,0,0,13,0,4,2,5,4,4,1,2,2,1,0,3,1,0,2,1,1,4,5,8,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,8,3,6,0,2007,0,0.0,460,62.0,40,953,1,2,0,0,0,953,0,60,953,694,1,5,1647,1999,0,2000,0,2,0,7,7917,1999.0,1,0,0,0,0,1,...,0,0,0,0,0,4,1,1,12,1,2,6,3,4,5,3,0,13,0,4,3,5,4,4,1,2,2,2,0,3,1,3,2,1,1,4,5,8,0,2
1456,349,2,3,6,163,2010,0,119.0,500,85.0,0,2073,2,2,0,790,0,589,0,20,1542,0,0,6,2073,1978,0,1988,1,2,0,7,13175,1978.0,1,0,0,0,0,1,...,0,0,0,0,0,4,1,1,9,1,2,2,3,4,0,3,0,10,4,4,3,4,4,4,1,2,1,3,0,3,1,3,2,1,2,4,2,8,0,2
1457,0,5,4,7,0,2010,0,0.0,252,66.0,60,1188,2,2,0,275,0,877,0,70,1152,1152,0,9,2340,1941,0,2006,0,1,0,9,9042,1941.0,1,2500,0,0,0,1,...,0,0,0,0,0,4,1,1,5,1,3,6,2,2,2,3,0,5,0,4,0,5,4,2,1,2,4,2,0,1,1,3,2,1,1,4,5,8,0,2
1458,366,4,2,5,1029,2010,112,0.0,240,68.0,0,1078,0,1,0,49,0,0,0,20,1078,0,0,6,1078,1950,0,1996,1,1,0,5,9717,1950.0,1,0,0,0,0,1,...,0,0,0,0,0,4,1,1,8,1,3,6,2,4,2,2,0,8,2,0,3,4,4,-1,1,2,1,2,0,3,1,3,2,3,2,4,2,8,0,2


In [ ]:
y=data[target]
y

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000
...,...
1455,175000
1456,210000
1457,266500
1458,142125


In [ ]:
# All datasets will become a matrix
X=X.to_numpy()
X

array([[  0.,   2.,   3., ...,   8.,   0.,   2.],
       [298.,   5.,   3., ...,   8.,   0.,   2.],
       [  0.,   9.,   3., ...,   8.,   0.,   2.],
       ...,
       [  0.,   5.,   4., ...,   8.,   0.,   2.],
       [366.,   4.,   2., ...,   8.,   0.,   2.],
       [736.,   6.,   3., ...,   8.,   0.,   2.]])

In [ ]:
y=y.to_numpy()
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [ ]:
X.shape

(1460, 110)

In [ ]:
y.shape

(1460, 1)

In [ ]:
X=StandardScaler().fit_transform(X)
y=StandardScaler().fit_transform(y)

In [ ]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [ ]:
X.shape

(1460, 110)

PCA Dimensionality Reduction

In [ ]:
pca=PCA(n_components=2)
p_components=pca.fit_transform(X)

In [ ]:
p_components

array([[ 2.40853487, -1.3304114 ],
       [ 0.43196109,  2.62314865],
       [ 2.82844318, -0.85553068],
       ...,
       [ 1.25987909, -0.86311838],
       [-2.83066434,  3.18197186],
       [-0.75664659,  2.68263272]])

In [ ]:
pca.explained_variance_ratio_

array([0.10207313, 0.04345606])

In [ ]:
pca_var=PCA(0.9)
new_pcs=pca_var.fit_transform(X)

In [ ]:
pca_var.n_components_

70

In [ ]:
pca.explained_variance_ratio_

array([0.10207313, 0.04345606])

In [ ]:
p_components=pd.DataFrame(p_components,columns=['PC1','PC2'])

In [ ]:
y=pd.DataFrame(y,columns=['y'])

In [ ]:
new_data=pd.concat([p_components,y],axis=1)

In [ ]:
new_data

,PC1,PC2,y
0,2.408535,-1.330411,0.347273
1,0.431961,2.623149,0.007288
2,2.828443,-0.855531,0.536154
3,-1.364218,0.550650,-0.515281
4,5.311988,-0.223575,0.869843
...,...,...,...
1455,1.306602,-2.289615,-0.074560
1456,1.095834,3.178839,0.366161
1457,1.259879,-0.863118,1.077611
1458,-2.830664,3.181972,-0.488523
